In [4]:
import tensorflow as tf

2025-05-10 16:57:14.849197: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 16:57:14.962311: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 16:57:15.059126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746917835.162338 1215127 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746917835.191238 1215127 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746917835.385434 1215127 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [69]:
# parse single tf record
def parse_tfrecord(file):
    feature_description={
        'note' : tf.io.FixedLenFeature([], tf.int64),
        'note_str' : tf.io.FixedLenFeature([], tf.string),
        'instrument' : tf.io.FixedLenFeature([], tf.int64),
        'instrument_str' : tf.io.FixedLenFeature([], tf.string),
        'pitch': tf.io.FixedLenFeature([], tf.int64),
        'velocity': tf.io.FixedLenFeature([], tf.int64),
        'sample_rate': tf.io.FixedLenFeature([], tf.int64),
        'audio': tf.io.FixedLenFeature([64000], tf.float32),
        'qualities': tf.io.FixedLenFeature([10], tf.int64),
        'qualities_str': tf.io.VarLenFeature(tf.string),
        'instrument_family': tf.io.FixedLenFeature([], tf.int64),
        'instrument_family_str': tf.io.FixedLenFeature([], tf.string),
        'instrument_source': tf.io.FixedLenFeature([], tf.int64),
        'instrument_source_str': tf.io.FixedLenFeature([], tf.string)
    }
    
    return tf.io.parse_single_example(file, feature_description)

# test parse function
# intake training dataset
trainDS_raw = tf.data.TFRecordDataset("/home/case/Documents/nsynth-train.tfrecord")

# map parsing function to dataset
trainingDS = trainDS_raw.map(parse_tfrecord)

num_examples = 0
#for _ in trainingDS:
#    num_examples += 1
#print(f"Total number of examples in dataset: {num_examples}")

# grab a few examples and examine them

tenExamples = trainingDS.take(10)

for i, example in enumerate(tenExamples):
    print(f"Example {i+1}")
    print(f"Note: {example['note']}")
    instrument_bytes = example['instrument_str'].numpy()
    print(f"Instrument: {instrument_bytes.decode('utf-8')}")
    instrument_fam_bytes = example['instrument_family_str'].numpy()
    print(f"Instrument Family: {instrument_fam_bytes.decode('utf-8')}")
    qualities_str_list_bytes = example['qualities_str'].values
    qualities_str_list = [value.decode('utf-8') for value in qualities_str_list_bytes.numpy()]
    print(f"Qualities: {qualities_str_list}")
    


Example 1
Note: 217495
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay', 'percussive']
Example 2
Note: 266066
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay', 'percussive']
Example 3
Note: 163673
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay']
Example 4
Note: 22459
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay']
Example 5
Note: 231254
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay']
Example 6
Note: 209097
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay', 'percussive']
Example 7
Note: 272473
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay', 'percussive']
Example 8
Note: 231913
Instrument: bass_synthetic_018
Instrument Family: bass
Qualities: ['distortion', 'fast_decay']
E